In [1]:
import zipfile
import os

def extract_files(zip_path):
    extract_dir = os.path.dirname(zip_path)

    with zipfile.ZipFile(zip_path, 'r') as zip_ref:
        zip_ref.extractall(extract_dir)
    print(f"Extracted all files from '{zip_path}' into '{extract_dir}'.")

extract_files("archive.zip")

Extracted all files from 'archive.zip' into ''.


In [7]:
import os
from torch.utils.data import Dataset, DataLoader
from PIL import Image
import torch
from torchvision import transforms
import torch.nn as nn
import torch.optim as optim
from torchvision import models
from tqdm import tqdm


In [9]:

# Custom Dataset class
class FireDataset(Dataset):
    def __init__(self, fire_dir, no_fire_dir, transform=None):
        self.fire_dir = fire_dir
        self.no_fire_dir = no_fire_dir
        self.transform = transform
        self.images = []
        self.labels = []

        # Load fire images
        for img_name in os.listdir(fire_dir):
            if img_name.lower().endswith(('.png', '.jpg', '.jpeg')):
                self.images.append(os.path.join(fire_dir, img_name))
                self.labels.append(1)  # Label 1 for fire

        # Load no-fire images
        for img_name in os.listdir(no_fire_dir):
            if img_name.lower().endswith(('.png', '.jpg', '.jpeg')):
                self.images.append(os.path.join(no_fire_dir, img_name))
                self.labels.append(0)  # Label 0 for no-fire

    def __len__(self):
        return len(self.images)

    def __getitem__(self, idx):
        img_path = self.images[idx]
        label = self.labels[idx]
        image = Image.open(img_path).convert('RGB')

        if self.transform:
            image = self.transform(image)

        return image, label


In [6]:
# Directories for fire and no-fire images
fire_dir = 'FLAME 3 CV Dataset (Sycan Marsh)/Fire/RGB/Corrected FOV/'
no_fire_dir = 'FLAME 3 CV Dataset (Sycan Marsh)/No Fire/RGB/Corrected FOV/'

# Transformations
transform = transforms.Compose([
    transforms.Resize((224, 224)),
    transforms.ToTensor(),
    transforms.Normalize(mean=[0.485, 0.456, 0.406], std=[0.229, 0.224, 0.225])
])

# Create the dataset
dataset = FireDataset(fire_dir, no_fire_dir, transform=transform)

# Split into training and validation datasets
train_size = int(0.75 * len(dataset))
val_size = len(dataset) - train_size
train_dataset, val_dataset = torch.utils.data.random_split(dataset, [train_size, val_size])

# Data loaders
batch_size = 16
train_loader = DataLoader(train_dataset, batch_size=batch_size, shuffle=True)
val_loader = DataLoader(val_dataset, batch_size=batch_size, shuffle=False)

# Check if the dataset is loaded correctly
print(f"Total images: {len(dataset)}")
print(f"Training images: {len(train_dataset)}")
print(f"Validation images: {len(val_dataset)}")

Total images: 738
Training images: 553
Validation images: 185


In [8]:
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
print(f"Using device: {device}")

# Load pre-trained ResNet50 model
model = models.resnet50(pretrained=True)
num_features = model.fc.in_features
model.fc = nn.Linear(num_features, 2)  # Adjust the final layer for binary classification
model = model.to(device)

# Loss function and optimizer
criterion = nn.CrossEntropyLoss()
optimizer = optim.Adam(model.parameters(), lr=0.001)

Using device: cuda


/home/dbhatka/.local/lib/python3.11/site-packages/torchvision/models/_utils.py:208: UserWarning: The parameter 'pretrained' is deprecated since 0.13 and may be removed in the future, please use 'weights' instead.
  warnings.warn(
/home/dbhatka/.local/lib/python3.11/site-packages/torchvision/models/_utils.py:223: UserWarning: Arguments other than a weight enum or `None` for 'weights' are deprecated since 0.13 and may be removed in the future. The current behavior is equivalent to passing `weights=ResNet50_Weights.IMAGENET1K_V1`. You can also use `weights=ResNet50_Weights.DEFAULT` to get the most up-to-date weights.
  warnings.warn(msg)


In [11]:
def train_model(model, train_loader, val_loader, criterion, optimizer, num_epochs=10):
    for epoch in range(num_epochs):
        print(f"Epoch {epoch + 1}/{num_epochs}")
        print('-' * 10)

        # Training phase
        model.train()
        running_loss = 0.0
        correct = 0
        total = 0

        for images, labels in tqdm(train_loader, desc="Training", leave=False):
            images, labels = images.to(device), labels.to(device)

            # Zero the parameter gradients
            optimizer.zero_grad()

            # Forward pass
            outputs = model(images)
            loss = criterion(outputs, labels)
            running_loss += loss.item()

            # Backward pass and optimization
            loss.backward()
            optimizer.step()

            # Track accuracy
            _, predicted = torch.max(outputs, 1)
            correct += (predicted == labels).sum().item()
            total += labels.size(0)

        train_loss = running_loss / len(train_loader)
        train_accuracy = correct / total * 100

        # Validation phase
        model.eval()
        val_loss = 0.0
        correct = 0
        total = 0

        with torch.no_grad():
            for images, labels in tqdm(val_loader, desc="Validation", leave=False):
                images, labels = images.to(device), labels.to(device)

                # Forward pass
                outputs = model(images)
                loss = criterion(outputs, labels)
                val_loss += loss.item()

                # Track accuracy
                _, predicted = torch.max(outputs, 1)
                correct += (predicted == labels).sum().item()
                total += labels.size(0)

        val_loss /= len(val_loader)
        val_accuracy = correct / total * 100

        print(f"Train Loss: {train_loss:.4f} | Train Acc: {train_accuracy:.2f}%")
        print(f"Val Loss: {val_loss:.4f} | Val Acc: {val_accuracy:.2f}%\n")


In [12]:
# Train the model
train_model(model, train_loader, val_loader, criterion, optimizer, num_epochs=20)

Epoch 1/20
----------


Train Loss: 0.1664 | Train Acc: 94.76%
Val Loss: 0.2171 | Val Acc: 94.59%

Epoch 2/20
----------


Train Loss: 0.1049 | Train Acc: 95.84%
Val Loss: 0.9436 | Val Acc: 92.43%

Epoch 3/20
----------


Train Loss: 0.0847 | Train Acc: 97.65%
Val Loss: 0.0656 | Val Acc: 98.38%

Epoch 4/20
----------


Train Loss: 0.0390 | Train Acc: 98.37%
Val Loss: 0.0492 | Val Acc: 97.30%

Epoch 5/20
----------


...

Train Loss: 0.0301 | Train Acc: 98.37%
Val Loss: 0.0448 | Val Acc: 97.84%

Epoch 17/20
----------


Train Loss: 0.0204 | Train Acc: 99.46%
Val Loss: 0.0265 | Val Acc: 98.92%

Epoch 18/20
----------


Train Loss: 0.0653 | Train Acc: 97.65%
Val Loss: 0.0639 | Val Acc: 96.22%

Epoch 19/20
----------


Train Loss: 0.0424 | Train Acc: 98.55%
Val Loss: 0.0295 | Val Acc: 98.38%

Epoch 20/20
----------


Train Loss: 0.0409 | Train Acc: 98.19%
Val Loss: 0.0260 | Val Acc: 99.46%



In [13]:
torch.save(model.state_dict(), "flame3_resnet50.pth")